Une autre piste de résolution pour le problème d'un unique véhicule est l'algorithme du simplexe
Il consiste à résoudre des problème de la forme suivante
$$ \min_{Ax=b ,x \geq 0} c^Tx $$

On se place donc dans ce contexte
En réecrivant notre problème qui, rappelons-le, est le suivant

$$\left\{
\begin{array}{r c l}
\displaystyle\sum_{\substack{k=0 }}^{n-1} x_i\Delta t_i &=& (c_f-c_0)Q_{max}\\
x_0 &\leq& I_{max}\\
...\\
x_{n-1} &\leq& I_{max}
\end{array}
\right.
$$
Où $x_{i}$ est l'intensité discrétisée au i-ème pas de discrétisation.

On le réécrit donc

$$\left\{
\begin{array}{r c l}
\displaystyle\sum_{\substack{k=0 }}^{n-1} x_i\Delta t_i &=& (c_f-c_0)Q_{max}\\
x_0 + y_0 &=& I_{max}\\
...\\
x_{n-1}+y_{n-1} &=& I_{max}
\end{array}
\right.
$$

Où $\forall i, y_{i} \geq 0$

On est donc rammené au problème voulu où le vecteur $x$ est composé des $x_i$ et des $y_i$ et est donc de dimension $2n$.

On implémente ce système dans la cellule suivante.


In [2]:
def A(n,t0,tf,PrixHoraire,Im,cf,c0,ph,pb):
    listet=np.linspace(t0,tf,n+1)
    Af=np.zeros((n+1,2*n))
    Af[0][0:n]=np.array([listet[i+1]-listet[i] for i in range (n)])
    for i in range(n):
        Af[i+1][i]=1
        Af[i+1][i+n]=1
    return Af

def B(n,t0,tf,PrixHoraire,Im,cf,c0):
    b = (cf-c0)*Qm
    B=np.zeros(n+1)
    B[0]=b
    for i in range(1,n+1):
        B[i]=Im
    return B

def C(n,t0,tf,PrixHoraire,Im,ph,pb):
    g=np.zeros(2*n)
    listet=np.linspace(t0,tf,n+1)
    for i in range (n):
        p=PrixHoraire(listet[i],ph,pb)
        q=(listet[i+1]-listet[i])
        g[i]=p*q
    return g

Détaillons maintenant la méthode du simplexe.
A est une matrice de taille $(n+1,2n)$. On pose $m=n+1$ et $p=2n$
On note $B$ une sous matrice inversible de la matrice $A$ de taille $(m,m)$

On note $iB$ les indices des colonnes de $A$ composant $B$ et $iN$ les autres indice. 

On pose $x_B=(x_i)_{i \in iB}$ et $x_N=(x_i)_{i \in iN}$.

On fait de même pour le vecteur $c^T$ en $c_B^T$ $c_N^T$.

Une solution quelconque de l'équation $Ax=b$ est x tel que
$$x_B=B^{-1}b$$
et $$x_N=0$$

On a ensuite $$Ax=b$$
qui donne
$$Bx_B+Nx_N=b$$
Soit
$$x_B=B^{-1}b-B^{-1}Nx_N$$
Puis la fonction coût s'écrit
$$c^Tx=c_B^Tx_B+c_N^Tx_N$$
soit en réinjectant l'expression de $x_N$
$$c^Tx = c_B^TB^{-1}b+(c_N^T-c_B^TB^{-1}N)x_N$$

On notera dans toutes la suite le vecteur $$\tilde{c_N}=c_N^T-c_B^TB^{-1}N$$
Dont les coefficients sont les $$\tilde{c_{N_i}}=c_{N_i}^T-c_B^TB^{-1}A^i$$ pour $i \in iN$ et où $A^i$ est la i-ème colonne de $A$

L'algorithme du simplexe consiste à remplacer les indices de $iB$ par ceux de $iN$  pour arriver à un $x$ minimisant $c^Tx$ dont l'extremisation est déterminée par les conditions de KKT. Pour cela on procède de la manière suivante

1) On choisit $i \in iN$ tel que $\tilde{c_{N_i}} < 0$ qui va rentrer dans la base $iB$ s'il n'y en pas la solution est optimale. On prend donc le plus grand i.e
$$i=\arg \max_{k \in iN} \tilde{c_{N_k}}$$

2) On choisit $j\in iB$ qui va sortir. Pour cela on choisit j tel que $x_B=B^{-1}b-B^{-1}A^it$ s'annule lorsque t augmente on a donc
$$ j= \arg_{j \in iB} \min_{(B^{-1}A^i)_{j \in iB}>0}\frac{(B^{-1}b)_j}{(B^{-1}A^i)_j} $$

3) On met à jour $iB$, $iN$, $B$ et $N$ puis on recalcule $x_N$ et $x_B$

4) Et on recommence ! Jusqu'a ce que $\forall i \in iN, c_{N_i} \geq 0$

On donne le code ci-dessous


In [ ]:
def simplexe(A,b,c,permut):
    # m: nombre de contraintes (vaut n+1 dans notre problème)
    # p: nombres d'inconnues (2n dans notre problème)
    #permut désigne la liste des indices de colonnes: les m premiers corresondent à iB les autres à iN
    m,p = A.shape
    if m>=p or c.shape[0] != p or b.shape[0] != m:
        return 'dimensions incompatibles'

    if min(b) < 0:
        return 'le vecteur b doit être >=0'

    while True :
        # matrice des colonnes
        Ap=np.column_stack((A[:,permut[i]] for i in range(p)))
        # coefficients de gains pour chacun des indices
        cp=np.array([c[permut[i]] for i in range(p)])

        # vérification de l'inversibilité
        if np.linalg.det(Ap[:,:m]) == 0:
            return 'matrice non inversible'

        #expression des la variable de base xB
        invAp=np.linalg.inv(Ap[:,:m])#Matrice B^(-1)
        Chb=np.dot(invAp,Ap[:,m:]) #Matrice B^(-1)*N
        xB=np.dot(invAp,b)

        # coefficients du gain dans les variables hors base
        cbase=cp[m:]-np.dot(cp[:m],Chb)#Ceci correspond au gain cn^T-cb^T*B^(-1)*N
        cmax=max(cbase)
        # si tout les coeffs sont  <0 on ne peut plus améliorer, 
        if cmax<=0:
            break #c'est donc la condition de sortie de la boucle
        # Etape 1
        # choix de la variable qui va rentrer on la note ihb (hors base)
        ihb=np.argmax(cbase)+m
        # Etape 2 
        # choix de la variable qui va sortir
        # On implemente la liste des (B^(-1)*b)j/(B^(-1)*A^i)j
        xrmax=np.array([xB[i]/Chb[i][ihb-m] for i in range(m)])
        #Et on va choisir ib comme défini dans la cellule ci-dessus
        res=0
        xrmax2=[]
        for i in range(m):
            if Chb[i][ihb-m]>0:
                xrmax2.append(xrmax[i])
        temp=np.min(xrmax2)
        for i in range(m):
            if xrmax[i]==temp:
                res=i
                break
        ib=res

        # actualisation de la permutation
        permut[ib],permut[ihb] = permut[ihb],permut[ib]
    # fin de l'algorithme
    # on complète le vecteur des variables avec xB pour les m premiers indices (iB) et des 0 après (iN)
    xp=np.hstack((xB,np.zeros(p-m)))
    # prise en compte de la permutation
    x=np.empty(p)
    for i in range(p):
        x[permut[i]]=xp[i]
    # renvoie la solution et le gain
    return x,np.dot(c,x)

In [3]:
# La fonction permuta initialise une base des colonnes
def permuta(n,t0,tf,PrixHoraire,Im,cf,c0):
    W=np.array([i for i in range (2*n)])
    return W

def TestSimplex(N,t0,tf,c0,cf,pb,ph,Im,PrixHoraire):
    #Résolution numérique
    A1=A(N,t0,tf,PrixHoraire,Im,cf,c0,ph,pb)
    B1=B(N,t0,tf,PrixHoraire,Im,cf,c0)
    C1=C(N,t0,tf,PrixHoraire,Im,ph,pb)
    l=permuta(N,t0,tf,PrixHoraire,Im,cf,c0)
    X=simplexe(A1,B1,C1,l)[0]
    #Attention le vecteur X se compose des variables d'écart y_i, il ne faut prendre que les N premières
    I=simplexe(A1,B1,C1,l)[0][:N] 
    Prix=simplexe(A1,B1,C1,l)[1]
    #partie graphique
    plt.figure()
    lt=np.linspace(t0,tf,N)
    lp=np.vectorize(PrixHoraire)(lt,ph,pb)
    lc=charge(I,t0,tf,c0)
    ly=fcout1(I,ph,pb,t0,tf,PrixHoraire)[1]
    plt.plot(lt,I,'+',label="Intensité")
    plt.plot(lt,ly,'+',label="Coût")
    plt.plot(lt,lp,'+',label="Prix horaire")
    plt.plot(lt,lc,'+',label="Charge")
    plt.legend()
    plt.show()
    return X

# Commentaires

-On constate une erreur que nous ne parvenons pas à expliquer concernant la première valeur de la liste des intensités. Celle-ci ne respecte pas l'exigence I>0 de la méthode du simplexe.
-Les autres valeurs de l'intensité au temps suivant semble respecter le caractère borné. En effet on constate que l'intensité reste entre Imax et 0 comme convenu.
-On constate également que la contrainte de charge est respectée malgré l'erreur commise sur la première valeur.